<h1 align="center">Senyals i Sistemes - Analitzador del senyal de veu<h1/>

In [1]:
import numpy as np
from scipy.fft import rfft, rfftfreq
from scipy.signal import windows,correlate,find_peaks
from scipy.io import wavfile
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.widgets import Slider, RangeSlider, RadioButtons, TextBox

NOM_FITXER = "Frase.wav"

class Segment:
    def __init__(self, offset=0, inici=0, final=1):
        self.offset = offset
        self.limits = (inici, final)

    @property
    def indexs(self):
        return self.offset + np.arange(self.limits[0], self.limits[1] + 1)
    
    @property
    def interval(self):
        return (self.offset+self.limits[0], self.offset+self.limits[1])

    def __len__(self):
        return self.limits[1] - self.limits[0] + 1

class Veu(object):
    def __init__(self, fitxer=NOM_FITXER, lon_fft=4096):
        # Llegim fitxer i normalitzem
        self.fitxer = fitxer
        self.fm, self.s = wavfile.read(fitxer)
        canals, = np.shape(np.shape(self.s))
        if canals > 1:
            self.s = np.average(self.s, axis=1)
        # Mètode 1:
        # self.s = self.s / np.max(self.s) / np.sqrt(2)
        # Mètode 2:
        self.s = self.s / 2**15

        # Calculem quants trams de 30 ms caben en aquest fragment de veu
        self.DURADA_SEG = 15e-3  # Durada dels segments d'anàlisi (segons)
        self.LONG_SEG = self.temps_a_mostra(self.DURADA_SEG)
        trams, exces = divmod(len(self.s), self.LONG_SEG)
        # Per fer el doble de trams solapats el 50%
        self.MAX_TRAMS = 2 * trams - 1

        # Definim vectors d'índexs i de temps
        self.n = np.arange(len(self.s))
        self.t = self.n / self.fm

        self.N = lon_fft
        self.finestra = "Hamming"

        self.S = Segment(offset=0, inici=0, final=self.LONG_SEG - 1)

    ###### Mètodes que manipulen segments
    def obten_dades(self, segment):
        """Retorna el vector de temps i les dades de veu corresponents a l'interval"""
        n = segment.indexs
        t = n / self.fm
        dades = self.s[n]
        dades = dades - np.mean(dades)
        return t, dades

    def interval(self, segment):
        """Retorna l'interval temporal del segment, en segons"""
        return tuple((segment.offset + np.array(segment.limits)) / self.fm)

    def obten_finestra(self, segment):
        if self.finestra == "Rectangular":
            return windows.boxcar(len(segment))
        elif self.finestra == "Hamming":
            return windows.hamming(len(segment))
        elif self.finestra == "Blackman-Harris":
            return windows.blackmanharris(len(segment))

    def temps_a_mostra(self, t):
        return int(t * self.fm)

    def mostra_a_temps(self, m):
        return m / self.fm

    def canvia_segment(self, num_tram):
        # Si solapem els trams, l'inici és cada LONG_SEG/2
        nou_inici_tram = num_tram * ( self.LONG_SEG >> 1 )  
        self.S.offset = nou_inici_tram

    def analitza_segment(self, num_tram):
        self.canvia_segment(num_tram)
        t, x = self.obten_dades(self.S)
        w = self.obten_finestra(self.S)
        senyal_enfinestrat = x*w
        R = correlate(senyal_enfinestrat, senyal_enfinestrat, mode='same', method='direct')
        R_max = np.max(R)
        R_norm = R / R_max     # Normalitzem
        
        #######################################################################################
        # Completeu el codi del dessota per tal d'assignar els valors correctes a les variables
        
        # «E» Ha de ser l'energia del tram de senyal
        E = R_max        #  <<<<<<<<<<< Completeu-ho
        
        # «maxims» ha de ser una estructura de dades que contingui 
        # les posicions i valors dels pics de l'autocorrelació del tram de senyal que estem analitzant
        #
        # Us suggerim d'utilitzar la funció «find_peaks» del paquet SciPy.Signal per tal d'omplir-la
        #
        # Aquesta funció té 2 paràmetres importants: «height» i «distance», que permeten controlar
        # quines condicions han de complir els màxims locals per ser considerats com a tals.
        indexs_pics, valors_pics = find_peaks(R_norm, distance=10, height = 0.5)    #  <<<<<<<<<<< Completeu-ho
        #
        # La funció «find_peaks» retorna una TUPLA amb 2 variables:
        # - Un array Numpy amb els índexs dels màxims detectats de l'autocorrelació
        # - Un diccionari amb un element 'peak_heights' que és un array NumPy amb els _valors_ dels màxims en els índexs anteriors
        # Es demana que extraieu aquesta informació i la poseu en 2 arrays NumPy diferents per tal que sigui més senzilla de manipular
        maxims = {                                    #  <<<<<<<<<<< Modificar-ho pels trams sonorsCompleteu-ho
            'indexs': np.array(indexs_pics),
            'valors': np.array(valors_pics['peak_heights'])
        }
        
        # A partir de la informació anterior, decidiu si el tram és sord o sonor
        sonor = len(maxims['indexs']) > 1 and E > 0.3              #  <<<<<<<<<<< Completeu-ho, ha de ser True o «False»
        
        # Finalment, cal determinar la freqüència «base» del fonema

        maxss = [x if x != 1 else 0 for x in maxims['valors'] ]
        maxind = np.argmax(maxss)
        
        N = 120 - maxims['indexs'][maxind] 
        T = N/self.fm
        f = 1/T if sonor else 0                                   #  <<<<<<<<<<< Completeu-ho

        # VALORS A RETORNAR:
        #   En tots els casos:
        #     - Vector de temps t
        #     - Senyal enfinestrat
        #     - L'autocorrelació normalitzada
        #     - L'energia del senyal
        #     - La sonoritat del tram
        #   Si el tram s'ha detectat com a SONOR
        #     - La freqüència «base» o fonamental del fonema
        #     - La informació dels màxims detectats
        #   Si el tram s'ha detectat com a SORD                        <<<<<<<<<<<<<<<<<  ATENCIÓ !
        #     - La freqüència «base»: es retornarà ZERO
        #     - La informació dels màxims detectats: es retornaran arrays BUITS (tal com estava definida originalment la variable «màxims»)
        return t, senyal_enfinestrat, R_norm, E, sonor, f, maxims
    

###########################################################################################################
    
###################################
##       Inici del programa      ##
###################################

# Inicialitzem el senyal de veu a analitzar
veu = Veu(NOM_FITXER)

###################################
##       Creació de la IGU       ##
###################################
# Fem la mida per defecte dels plots una mica més gran
plt.rcParams["figure.figsize"] = [9, 11.8]
plt.rcParams["figure.dpi"] = 80

# Creeem la figura amb les gràfiques que hi han d'anar
fig, (ax1, ax2, ax3) = plt.subplots(
    3, 1, gridspec_kw={"height_ratios": [0.7, 1, 1.1]}, constrained_layout=False
)
fig.subplots_adjust(top=0.95, bottom=0.2, hspace=0.25)

###### Gràfic superior
# Als eixos superiors hi pintem el segment complet
ax1.set_navigate(False)  # Desactivo Pan/Zoom/etc.
ax1.set_ylim(ymin=-1, ymax=1)
ax1.set_xlim(xmin=0, xmax=veu.n[-1] / veu.fm)
ax1.grid(False)
ax1.set_xlabel("Temps [s]")
ax1.set_ylabel("Senyal complet")
titol = ax1.set_title(
    "Senyals i Sistemes - Analitzador de senyals de veu", fontsize="x-large"
)
llegenda = "Fitxer {}, fm={} Hz".format(veu.fitxer, veu.fm)
(senyal,) = ax1.plot(veu.n / veu.fm, veu.s, lw=0.5, label=llegenda)
ax1.legend(loc="best")
# Pinto la finestra inicial de 30ms
rect_segment = mpatches.Rectangle((0, -1), 30e-3, 2, color="C1")
ax1.add_patch(rect_segment)

###### Gràfic central
# Als eixos centrals, el segment de 30 ms seleccionat per l'slider
ax2.set_navigate(False)
t_segment, x, R, E, S, f, max_info = veu.analitza_segment(0)
ax2.set_ylim(ymin=-1, ymax=1)
ax2.set_xlim(xmin=t_segment[0], xmax=t_segment[-1])
ax2.grid(True)
(segment,) = ax2.plot(t_segment, x, lw=1, color="C1")
ax2.set_xlabel("Temps [s]")
ax2.set_ylabel("Fragment de 30 ms")

###### Gràfic inferior
# Als eixos inferiors hi va l'autocorrelació del tram
# L'eix de temps és fix: vector de -15ms a +15ms amb T=1/fs
t_corr = 1000*(np.arange(0,len(R))/veu.fm - 15E-3)
ax3.set_xlim(xmin=t_corr[0], xmax=t_corr[-1])
ax3.set_ylim(ymin=-0.6, ymax=1)
ax3.grid(True)
(autocorr_segment,) = ax3.plot(t_corr, R, lw=1, color="C2", label="Autocorrelació del fragment de 30ms")
max_t = (max_info['indexs']-120)/8      # és (max_info[0]-centre)*1000/8000, expressat en milisegons
max_val = max_info['valors']
(maxims,) = ax3.plot(max_t, max_val, "o", color="C3", label="Màxims locals detectats")
ax3.legend(loc="best")
ax3.set_xlabel("Desplaçament [ms]")
ax3.set_ylabel("Autocorrelació del fragment (normalitzada)")

###### Controls
# Radiobuttons per escollir la finestra
axwindow = plt.axes([0.12, 0.03, 0.2, 0.1], frame_on=False)
axwindow.set_title("Finestra")
axwindow.set_frame_on(True)
window_buttons = RadioButtons(
    ax=axwindow,
    labels=["Rectangular", "Hamming", "Blackman-Harris",],
    active=1,
    activecolor="k",
)
# La funció que es cridarà quan canviem el tipus de finestra
def update_window(button):
    if (
        (button == "Rectangular")
        or (button == "Hamming")
        or (button == "Blackman-Harris")
    ):
        veu.finestra = button
    else:
        pass
    update_segment(tram_slider.val)
window_buttons.on_clicked(update_window)

# Etiquetes que indiquen la mostra inicial i final del tram escollit
mostres = "Índexs del tram:  Mostra inicial={ini:05d}  ,  Mostra final={fin:05d}"
axindexs = plt.axes([0.4, 0.05, 0.38, 0.05], frameon=False)
axindexs.set_axis_off()
txt = axindexs.text(0, 0, mostres.format(ini=0,fin=0))

# Faig un slider horitzontal per seleccionar còmodament el segment d'àudio a processar
axtram = plt.axes([0.4, 0.11, 0.45, 0.02])
axtram.set_title("Tram")
tram_slider = Slider(
    ax=axtram,
    label="",
    valmin=0,
    valmax=veu.MAX_TRAMS - 1,
    valinit=0,
    valstep=1,
    facecolor="C1",
)
# La funció que es cridarà a cada canvi de l'slider del tram
def update_segment(noutram):
    try:
        t, x, R, E, S, f, max_info = veu.analitza_segment(noutram)
        rect_segment.set(x=t[0])
        segment.set_xdata(t)
        segment.set_ydata(x)
        ax2.set_xlim(xmin=t[0], xmax=t[-1])
        autocorr_segment.set_ydata(R)
        max_t = (max_info['indexs']-120)/8      # és (max_info[0]-centre)*1000/8000, expressat en milisegons
        max_val = max_info['valors']
        maxims.set_xdata(max_t)
        maxims.set_ydata(max_val)
        idx = veu.S.interval
        T = mostres.format(ini=idx[0],fin=idx[1]) + "\n"
        T += "Energia: {:2.4f}\n".format(E)       
        if S == True:
            T += "Sonor -> "
        elif S == False:
            T += "Sord -> "
        else:
            T += "Sonoritat a determinar -> "
        T += "Freq. fonamental: "
        T += "{:3.2f} Hz".format(f) if f is not np.nan else "----"
        txt.set_text(T)
        fig.canvas.draw_idle()
    except:
        pass
tram_slider.on_changed(update_segment)

# Activem la detecció de tecles, per tal de poder seleccionar el tram amb els cursors
def on_keypressed(event):
    if event.key=='right':
        tram_slider.set_val(min(tram_slider.val + 1, tram_slider.valmax))
    elif event.key=='left':
        tram_slider.set_val(max(tram_slider.val - 1, tram_slider.valmin))
    else:
        pass
cid = fig.canvas.mpl_connect('key_press_event', on_keypressed)

plt.show()  # Mostrem la IGU i li cedim el control